In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import torchvision.models as models
import cv2


In [2]:
class Extractor(nn.Module):
    def __init__(self):
        super(Extractor, self).__init__()
        
        self.vgg = models.vgg19(pretrained=True)        
        self.vgg.classifier = nn.Sequential(
            *(self.vgg.classifier[i] for i in range(5)))
        
        
    def forward(self, images): 
        return self.vgg(images)

In [3]:
def read_img(img_path):
    img = plt.imread(img_path)
    img = cv2.resize(img,(224,224))
    img = img / 255
    img = img.transpose((2,0,1))
    return img
    

In [4]:
with open("./food-101/meta/train.txt","r") as f:
    training_list = f.readlines()
with open("./food-101/meta/test.txt","r") as f:
    test_list = f.readlines()
    
with open('./food-101/meta/labels.txt','r') as f:
    labels = f.readlines()
    
training_list = ["./food-101/images/"+ path[:-1] +".jpg" for path in training_list]
test_list = ["./food-101/images/"+ path[:-1] +".jpg" for path in test_list]
labels = [label[:-1].replace(" ",'_').lower() for label in labels]

In [5]:
vgg = Extractor()
vgg.cuda()
vgg.eval()

Extractor(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace)
      (16): Conv2d(256, 256, kernel_size=(3, 3)

In [6]:
import pickle

test_data = []
counter = 0
for i in range(len(test_list)):
    img = read_img(test_list[i])
    x = torch.FloatTensor(img).view(1,3,224,224)
    x = Variable(x.cuda())
    feature = vgg(x)
    label_id = labels.index(test_list[i].split("/")[-2])
    test_data.append((label_id,feature.cpu().data.numpy()))
#     if i % 10000 == 0:
#         with open('test_' + str(counter) + ".pkl","wb") as f:
#             pickle.dump(test_data, f)
#         print('test_' + str(counter) + ".pkl")
#         test_data = []
#         counter += 1

In [8]:
 with open("test_data_vgg19.pkl","wb") as f:
        pickle.dump(test_data, f)

In [9]:
len(test_data)

25250

In [7]:
import pickle

training_data = []
counter = 0
for i in range(len(training_list)):
    img = read_img(training_list[i])
    x = torch.FloatTensor(img).view(1,3,224,224)
    x = Variable(x.cuda())
    feature = vgg(x)
    label_id = labels.index(training_list[i].split("/")[-2])
    training_data.append((label_id,feature.cpu().data.numpy()))
    if i % 10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000


In [10]:
len(training_data)

75747

In [11]:
 with open("training_data_vgg19.pkl","wb") as f:
        pickle.dump(training_data, f)

In [29]:
training_data[-1]

(100, array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32))